# Movies Tonight Part 5 (optional & experimental)

__This tutorial demonstrates various techniques to extract and merge data from the Movies Tonight database and the [IMDB API](https://www.theimdbapi.org). Unlike the previous parts, there is nothing to do except study the code. Run the cells one at a time to see how they work.__

## Using Pandas's Built-in SQL Alchemy Support 

In [1]:
import pandas as pd

# initialize a Pandas database connection engine 
from sqlalchemy import create_engine
engine = create_engine("sqlite:///Movies1.db")

**1. Loading each table as a separate DataFrame.** This cell uses the Pandas [read_sql_table() function](https://pandas.pydata.org/pandas-docs/stable/io.html#sql-queries) to load each table as a DataFrame. Name each DataFrame to match the table name in the database (e.g., `movies` for the `MOVIES` table). You will need to specify the `index_col`

In [6]:
# Note: SQL Alchemy is case sensitive; the TABLE and Column names have to match exactly 

artists = pd.read_sql_table('ARTISTS',engine,index_col='ARTISTID') # index_col is needed to indicate the primary key column
credits = pd.read_sql_table('CREDITS',engine,index_col='CREDITID')
movies = pd.read_sql_table('MOVIES',engine,index_col='MOVIEID')
theaters = pd.read_sql_table('THEATERS',engine,index_col='THEATERID')
# shows = pd.read_sql_table('SHOWS',engine,index_col='SHOWID')

**2. A trial join of the movies, shows, and theaters DataFrames.** Uses [Pandas DataFrame's `join()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) to create a new DataFrame named `movie_shows`. 

In [9]:
# Note: Joins must always be from the FK (e.g., SHOWS.MID) to the PK (e.g., MOVIES.ID)
# We can chain joins, just like SQL. Below is a chained (shows --> movies --> theaters) join.

movie_shows = credits.join(movies,on='MOVIEID')
movie_shows

,CCODE,MOVIEID,ARTISTID,TITLE,RATING
CREDITID,,,,,
1,A,1,1,"Associate, The",PG-13
2,A,1,2,"Associate, The",PG-13
3,A,1,3,"Associate, The",PG-13
4,A,1,4,"Associate, The",PG-13
5,A,1,5,"Associate, The",PG-13
...,...,...,...,...,...
158,A,23,149,That Thing You Do,PG
159,A,23,150,That Thing You Do,PG
160,A,23,151,That Thing You Do,PG


## Using Embedded SQL with %sql Magic 

In [11]:
# standard imports for %sql magic
%load_ext sql

# initialize a %sql database connection; may have to adjust username and password 
%sql sqlite:///Movies1.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @Movies1.db'

**3. Another (movie --> show --> theater) join, this time using a one-line embedded `%sql` statement.** The %sql magic lbrary uses its own `ResultSet` data type for the results. Fortunately, `ResultSet` has a `DataFrame()` method to convert to a DataFrame.  

In [12]:
# A one-line %sql call
movie_shows_rs = %sql SELECT * FROM MOVIES JOIN SHOWS ON (MOVIES.ID=SHOWS.MID) JOIN THEATERS ON (SHOWS.TID=THEATERS.ID)
# Then a conversion to a DataFrame
movie_shows_df = movie_shows_rs.DataFrame()
movie_shows_df

 * sqlite:///Movies1.db
(sqlite3.OperationalError) no such column: MOVIES.ID
[SQL: SELECT * FROM MOVIES JOIN SHOWS ON (MOVIES.ID=SHOWS.MID) JOIN THEATERS ON (SHOWS.TID=THEATERS.ID)]
(Background on this error at: http://sqlalche.me/e/e3q8)


AttributeError: 'NoneType' object has no attribute 'DataFrame'

**4. The (movie --> show --> theater) join again, this time using a multi-line `%%sql` statement.** Note that we are using `%%sql` syntax to indicate that the query is on multiple lines. Since the cell is SQL and *not* Python, we can't assign the result directly to a Python variable. We'll use a neat Jupyter trick below. 

In [ ]:
%%sql
SELECT * 
FROM MOVIES 
    JOIN SHOWS ON (MOVIES.ID=SHOWS.MID) 
    JOIN THEATERS ON (SHOWS.TID=THEATERS.ID) 

**Now for the trick.** Jupyter store the results of the previous code cell in a special variable called `_` (underscore). We can then use that to create our DataFrame. 

In [ ]:
movie_shows_rs = _
movie_shows_df = movie_shows_rs.DataFrame()
movie_shows_df

## Using a RESTful API to Connect to a Remote Data Source

**5. A custom utility module to retrieve data from IMDB.** Inside this folder is the file `myimdbapiutl.py`. Open it in a text editor. The `imdb_import_movie()` function can retrieve IMDB data for any movie that is likely to appear in our database. We just have to know the (title, year) or imdb_id of the movie. Notice how the function is documented with a docstring and uses Python's exception handling to raise errors. 

In [ ]:
from myimdbapiutil import imdb_import_movie

**6. A few example calls.** The cells below call the `imdb_import_movie()` function for the movie 'Big Night' and display some of the output.

In [ ]:
# Called with the movie title and the year
big_night = imdb_import_movie(mtitle='Big Night',year=1996)

# Show the movie data
big_night['movie'].transpose() # turned side-ways so we can read it better.

In [ ]:
# Called with just the movie title (and without the year)
big_night = imdb_import_movie(mtitle='Big Night')

# Show the cast 
big_night['cast']

In [ ]:
# Called with just the imdb_id
big_night = imdb_import_movie(mid='tt0115678')

# Show the cast 
big_night['cast']

## Merging SQL Data with API Data

__For our last demonstration we will use the [Pandas `merge()` function](https://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging) to glue together data from multiple sources.__

**7. This is an example of an adaptor for using our `myimdbapiutil` API module.** We might consider moving it into `myimdbapiutil.py`, but is included here as a demo.

In [ ]:
def find_movie(mtitle, opt='movie'):
    movie = imdb_import_movie(mtitle=mtitle)
    return movie[opt]

**8. A little Pandas ninja magic.** This calls the ulility module to get a fresh copy of the IMDB data. While the service is pretty unreliable, we can always go back to our file backup if needed. (See the comments below.)

In [ ]:
# The following may fail if the service is down; if so then go to the next cell.
# Like the adaptor code above this could 
m=movies['TITLE'].apply(find_movie).tolist()

# After running this cell be patient; you should _eventually_ get a
# list of all 23 movies OR an error.

imdb_movies = pd.concat(m)
imdb_movies.set_index('imdb_id',inplace=True)
imdb_movies.to_csv("imdb_movies.csv") # update the csv file with the latest copy

In [ ]:
# uncomment below if the IMDB lookup fails
imdb_movies = pd.read_csv("imdb_movies.csv", index_col=0)
imdb_movies

**9. Prep to allow the join/merge.** We need to set up a common key for the two DataFrames. 

In [ ]:
# Add the mid to the imdb_movies DataFrame
imdb_movies['mid']=movies.index
imdb_movies

**10. Now for the merge.** You should probably [read the docs](https://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging) for this one. 

In [ ]:
movies_merged = pd.merge(movies,imdb_movies,left_index=True,right_on='mid')
movies_merged

**That's all for now. Hopefully this gives you plenty of ideas for your class projects.**